In [ ]:
# Notebook 3: Data Analysis for Streamlit Application

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys
import joblib
from pathlib import Path
from datetime import datetime
import glob

# Set up paths
parent_dir = str(Path().resolve().parent)
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Set up directories
processed_dir = os.path.normpath('../data/processed')
api_responses_dir = os.path.normpath('../data/api_responses')
streamlit_data_dir = os.path.normpath('../data/streamlit')

# Create streamlit data directory
os.makedirs(processed_dir, exist_ok=True)
os.makedirs(streamlit_data_dir, exist_ok=True)

print("📊 Solana DeFi Tracker - Analysis for Streamlit")
print(f"Processing data for Streamlit application")
print(f"Analysis timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

def load_latest_file(directory, pattern):
    """Load the most recent file matching the pattern"""
    files = glob.glob(os.path.join(directory, pattern))
    if not files:
        return None, None
    latest_file = max(files, key=os.path.getctime)
    return joblib.load(latest_file), latest_file

def load_and_convert_data(directory, pattern, data_name):
    """Load and convert data to DataFrame if necessary"""
    data, filepath = load_latest_file(directory, pattern)
    if data is not None:
        print(f"✅ Loaded {data_name}: {os.path.basename(filepath)}")
        # Keep dicts intact (e.g., CoinGecko enhanced data)
        if isinstance(data, dict):
            print(f"ℹ️ {data_name} is a dict (keeping as-is for specialized handling)")
            return data, filepath
        if isinstance(data, list):
            try:
                data = pd.DataFrame(data)
                print(f"✅ Converted {data_name} to DataFrame (Shape: {data.shape})")
                print(f"Columns: {data.columns.tolist()}")
            except Exception as e:
                print(f"❌ Error converting {data_name} to DataFrame: {e}")
                data = pd.DataFrame()
        elif isinstance(data, pd.DataFrame):
            print(f"✅ Loaded {data_name} (Shape: {data.shape})")
            print(f"Columns: {data.columns.tolist()}")
        else:
            print(f"❌ Unexpected {data_name} type: {type(data)}")
            data = pd.DataFrame()
    else:
        print(f"❌ No {data_name} found")
        data = pd.DataFrame()
    return data, filepath

def format_currency(amount):
    """Format currency amount with appropriate units"""
    if amount is None or amount == 0:
        return "$0"
    if amount >= 1_000_000_000:
        return f"${amount/1_000_000_000:.2f}B"
    elif amount >= 1_000_000:
        return f"${amount/1_000_000:.2f}M"
    elif amount >= 1_000:
        return f"${amount/1_000:.2f}K"
    else:
        return f"${amount:.2f}"

print("\n🔍 Loading Collected Data...")

# Load all data with type checking
tvl_data, tvl_file = load_and_convert_data(api_responses_dir, 'solana_defi_tvl_*.joblib', 'TVL data')
revenue_data, revenue_file = load_and_convert_data(api_responses_dir, 'solana_revenue_*.joblib', 'Revenue data')
fees_data, fees_file = load_and_convert_data(api_responses_dir, 'solana_fees_*.joblib', 'Fees data')
coingecko_data, coingecko_file = load_and_convert_data(api_responses_dir, 'solana_coingecko_enhanced_*.joblib', 'CoinGecko data')

# Handle CoinGecko data specifically (supports dict/list/df)
coingecko_df = pd.DataFrame()
if coingecko_data is not None:
    if isinstance(coingecko_data, dict):
        # Expect keys per protocol; convert to rows
        coingecko_df = pd.DataFrame.from_dict(coingecko_data, orient='index').reset_index()
        coingecko_df.rename(columns={'index': 'protocol_key'}, inplace=True)
        print(f"✅ Converted CoinGecko dict to DataFrame (Shape: {coingecko_df.shape})")
    elif isinstance(coingecko_data, list):
        coingecko_df = pd.DataFrame(coingecko_data)
        print(f"✅ Converted CoinGecko list to DataFrame (Shape: {coingecko_df.shape})")
    elif isinstance(coingecko_data, pd.DataFrame):
        coingecko_df = coingecko_data.copy()
        print(f"✅ CoinGecko DataFrame shape: {coingecko_df.shape}")
    else:
        print("❌ Unsupported CoinGecko data type after load; skipping.")
else:
    print("❌ No CoinGecko data found")

holders_data, holders_file = load_and_convert_data(api_responses_dir, 'solana_token_holders_*.joblib', 'Token Holders data')

print("\n🔄 Processing and Consolidating Data...")

def consolidate_protocol_data():
    """Consolidate all collected data into unified DataFrames for Streamlit"""
    if isinstance(tvl_data, pd.DataFrame) and not tvl_data.empty:
        df_base = tvl_data.copy()
    else:
        print("❌ No TVL data to process")
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

    # Normalized key
    df_base['protocol_clean'] = df_base['name'].astype(str).str.lower().str.replace(' ', '_')

    print(f"📊 Base TVL data: {len(df_base)} protocols")

    # ---- Revenue (rename before merging)
    if isinstance(revenue_data, pd.DataFrame) and not revenue_data.empty:
        revenue_clean = revenue_data.copy()
        if 'protocol' in revenue_clean.columns:
            revenue_clean['protocol_clean'] = revenue_clean['protocol'].astype(str).str.lower().str.replace(' ', '_')
        elif 'protocol_name' in revenue_clean.columns:
            revenue_clean['protocol_clean'] = revenue_clean['protocol_name'].astype(str).str.lower().str.replace(' ', '_')
        else:
            # fallback: try name
            revenue_clean['protocol_clean'] = revenue_clean.get('name', '').astype(str).str.lower().str.replace(' ', '_')

        # rename columns up-front
        rename_rev = {}
        if 'total_24h' in revenue_clean.columns: rename_rev['total_24h'] = 'revenue_24h'
        if 'total_7d' in revenue_clean.columns:  rename_rev['total_7d']  = 'revenue_7d'
        if 'total_30d' in revenue_clean.columns: rename_rev['total_30d'] = 'revenue_30d'
        revenue_clean.rename(columns=rename_rev, inplace=True)

        cols_rev = ['protocol_clean'] + [c for c in ['revenue_24h','revenue_7d','revenue_30d'] if c in revenue_clean.columns]
        df_base = df_base.merge(revenue_clean[cols_rev].drop_duplicates('protocol_clean'), on='protocol_clean', how='left')
        print(f"✅ Merged revenue data: {revenue_clean.shape[0]} rows available")

    # ---- Fees (rename before merging)
    if isinstance(fees_data, pd.DataFrame) and not fees_data.empty:
        fees_clean = fees_data.copy()
        if 'protocol' in fees_clean.columns:
            fees_clean['protocol_clean'] = fees_clean['protocol'].astype(str).str.lower().str.replace(' ', '_')
        elif 'protocol_name' in fees_clean.columns:
            fees_clean['protocol_clean'] = fees_clean['protocol_name'].astype(str).str.lower().str.replace(' ', '_')
        else:
            fees_clean['protocol_clean'] = fees_clean.get('name', '').astype(str).str.lower().str.replace(' ', '_')

        # rename columns up-front
        rename_fees = {}
        if 'total_24h' in fees_clean.columns: rename_fees['total_24h'] = 'fees_24h'
        if 'total_7d' in fees_clean.columns:  rename_fees['total_7d']  = 'fees_7d'
        if 'total_30d' in fees_clean.columns: rename_fees['total_30d'] = 'fees_30d'
        fees_clean.rename(columns=rename_fees, inplace=True)

        cols_fees = ['protocol_clean'] + [c for c in ['fees_24h','fees_7d','fees_30d'] if c in fees_clean.columns]
        df_base = df_base.merge(fees_clean[cols_fees].drop_duplicates('protocol_clean'), on='protocol_clean', how='left')
        print(f"✅ Merged fees data: {fees_clean.shape[0]} rows available")

    # ---- CoinGecko (market/price/volume/rank)
    if not coingecko_df.empty:
        cg = coingecko_df.copy()
        # derive protocol_clean
        if 'protocol_name' in cg.columns:
            cg['protocol_clean'] = cg['protocol_name'].astype(str).str.lower().str.replace(' ', '_')
        elif 'protocol_key' in cg.columns:
            cg['protocol_clean'] = cg['protocol_key'].astype(str).str.lower().str.replace(' ', '_')
        elif 'name' in cg.columns:
            cg['protocol_clean'] = cg['name'].astype(str).str.lower().str.replace(' ', '_')
        else:
            # if no obvious key, skip merge
            cg['protocol_clean'] = np.nan

        keep_cols = ['protocol_clean','symbol','current_price_usd','market_cap_usd',
                     'total_volume_24h_usd','price_change_24h_percent','price_change_7d_percent',
                     'price_change_30d_percent','circulating_supply','total_supply','market_cap_rank']
        keep_cols = [c for c in keep_cols if c in cg.columns]
        if 'protocol_clean' in keep_cols:
            df_base = df_base.merge(cg[keep_cols].dropna(subset=['protocol_clean']).drop_duplicates('protocol_clean'),
                                    on='protocol_clean', how='left')
            print(f"✅ Merged CoinGecko data: {cg.shape[0]} rows available")
        else:
            print("⚠️ CoinGecko data missing protocol key; skipped merge")

    # ---- Derived metrics (safe)
    df_base['revenue_24h']    = df_base.get('revenue_24h', 0).fillna(0)
    df_base['fees_24h']       = df_base.get('fees_24h', 0).fillna(0)
    df_base['market_cap_usd'] = df_base.get('market_cap_usd', 0).fillna(0)
    df_base['tvl']            = df_base.get('tvl', 0).fillna(0)

    # Price-to-Fees ratio (annualized)
    df_base['pf_ratio'] = np.where(
        df_base['fees_24h'] > 0,
        df_base['market_cap_usd'] / (df_base['fees_24h'] * 365),
        np.nan
    )
    # Price-to-Revenue ratio (annualized)
    df_base['pr_ratio'] = np.where(
        df_base['revenue_24h'] > 0,
        df_base['market_cap_usd'] / (df_base['revenue_24h'] * 365),
        np.nan
    )
    # Market Cap to TVL ratio
    df_base['mcap_tvl_ratio'] = np.where(
        df_base['tvl'] > 0,
        df_base['market_cap_usd'] / df_base['tvl'],
        np.nan
    )

    print(f"📊 Final consolidated dataset: {df_base.shape}")
    return df_base

def process_token_holders_data():
    """Process token holders data for concentration analysis"""
    if not (isinstance(holders_data, pd.DataFrame) and not holders_data.empty):
        print("❌ No token holders data available")
        return pd.DataFrame()

    print(f"📊 Processing token holders data: {holders_data.shape}")
    # Ensure sorted by rank so head() picks top holders
    token_df_sorted = holders_data.sort_values(['token_name','rank'], ascending=[True, True]).copy()

    concentration_metrics = []
    for token_name in token_df_sorted['token_name'].unique():
        token_df = token_df_sorted[token_df_sorted['token_name'] == token_name]
        if len(token_df) > 0:
            total_supply = token_df['ui_amount'].sum()
            top_1_share  = (token_df.iloc[0]['ui_amount'] / total_supply * 100) if total_supply > 0 else 0
            top_5_share  = (token_df.head(5)['ui_amount'].sum() / total_supply * 100) if total_supply > 0 else 0
            top_10_share = (token_df.head(10)['ui_amount'].sum() / total_supply * 100) if total_supply > 0 else 0

            # Gini coefficient
            balances = token_df['ui_amount'].values
            balances = balances[balances > 0]
            if len(balances) > 1:
                n = len(balances)
                mean_balance = np.mean(balances)
                if mean_balance > 0:
                    diffs = np.abs(balances.reshape(-1,1) - balances.reshape(1,-1))
                    gini = diffs.sum() / (2 * n * n * mean_balance)
                else:
                    gini = 0
            else:
                gini = 0

            concentration_metrics.append({
                'token_name': token_name,
                'token_symbol': token_df.iloc[0].get('token_symbol', ''),
                'total_accounts_analyzed': len(token_df),
                'top_1_holder_share': top_1_share,
                'top_5_holders_share': top_5_share,
                'top_10_holders_share': top_10_share,
                'gini_coefficient': gini,
                'largest_holder_amount': token_df.iloc[0]['ui_amount'] if len(token_df) > 0 else 0
            })

    concentration_df = pd.DataFrame(concentration_metrics)
    print(f"✅ Processed concentration metrics for {len(concentration_df)} tokens")
    return concentration_df

def create_streamlit_datasets():
    """Create optimized datasets for each Streamlit tab"""
    df_overview = consolidate_protocol_data()

    # Tab 1: Protocol Overview (TVL, Market Cap, Categories)
    if not df_overview.empty:
        overview_cols = [
            'name', 'category', 'tvl', 'market_cap_usd', 'current_price_usd',
            'change_1d', 'price_change_24h_percent', 'mcap_tvl_ratio'
        ]
        # ensure columns exist
        for c in ['market_cap_usd','current_price_usd','change_1d','price_change_24h_percent','mcap_tvl_ratio']:
            if c not in df_overview.columns:
                df_overview[c] = np.nan

        df_tab1 = df_overview[overview_cols].copy()
        df_tab1.rename(columns={
            'name': 'Protocol',
            'category': 'Category',
            'tvl': 'TVL_USD',
            'market_cap_usd': 'Market_Cap_USD',
            'current_price_usd': 'Price_USD',
            'change_1d': 'TVL_Change_1d',
            'price_change_24h_percent': 'Price_Change_24h',
            'mcap_tvl_ratio': 'MCap_TVL_Ratio'
        }, inplace=True)

        # Fill NaN values
        df_tab1['Market_Cap_USD'] = df_tab1['Market_Cap_USD'].fillna(0)
        df_tab1['Price_USD'] = df_tab1['Price_USD'].fillna(0)
        df_tab1['TVL_Change_1d'] = df_tab1['TVL_Change_1d'].fillna(0)
        df_tab1['Price_Change_24h'] = df_tab1['Price_Change_24h'].fillna(0)

        # Sort by TVL
        df_tab1 = df_tab1.sort_values('TVL_USD', ascending=False).reset_index(drop=True)
        print(f"📋 Tab 1 (Overview) dataset: {df_tab1.shape}")
    else:
        df_tab1 = pd.DataFrame()

    # Tab 2: Financial Metrics (Revenue, Fees, Ratios)
    if not df_overview.empty:
        for c in ['revenue_24h','revenue_7d','revenue_30d','fees_24h','fees_7d','fees_30d','pf_ratio','pr_ratio','market_cap_usd','tvl']:
            if c not in df_overview.columns:
                df_overview[c] = 0

        financial_cols = [
            'name', 'category', 'tvl', 'revenue_24h', 'revenue_7d', 'revenue_30d',
            'fees_24h', 'fees_7d', 'fees_30d', 'market_cap_usd', 'pf_ratio', 'pr_ratio'
        ]
        df_tab2 = df_overview[financial_cols].copy()
        df_tab2.rename(columns={
            'name': 'Protocol',
            'category': 'Category',
            'tvl': 'TVL_USD',
            'revenue_24h': 'Revenue_24h',
            'revenue_7d': 'Revenue_7d',
            'revenue_30d': 'Revenue_30d',
            'fees_24h': 'Fees_24h',
            'fees_7d': 'Fees_7d',
            'fees_30d': 'Fees_30d',
            'market_cap_usd': 'Market_Cap_USD',
            'pf_ratio': 'PF_Ratio',
            'pr_ratio': 'PR_Ratio'
        }, inplace=True)

        # Fill NaN values
        for col in ['Revenue_24h','Revenue_7d','Revenue_30d','Fees_24h','Fees_7d','Fees_30d','PF_Ratio','PR_Ratio','Market_Cap_USD','TVL_USD']:
            df_tab2[col] = df_tab2[col].fillna(0)

        # Filter out protocols with no financial data
        df_tab2 = df_tab2[
            (df_tab2['Revenue_24h'] > 0) |
            (df_tab2['Fees_24h'] > 0) |
            (df_tab2['Market_Cap_USD'] > 0)
        ].reset_index(drop=True)
        print(f"📋 Tab 2 (Financial) dataset: {df_tab2.shape}")
    else:
        df_tab2 = pd.DataFrame()

    # Tab 3: Token Distribution Analysis
    df_tab3 = process_token_holders_data()
    if not df_tab3.empty:
        print(f"📋 Tab 3 (Token Distribution) dataset: {df_tab3.shape}")

    return df_tab1, df_tab2, df_tab3, df_overview

# Execute data consolidation
df_tab1, df_tab2, df_tab3, df_full = create_streamlit_datasets()

print("\n📈 Creating Analysis Summaries...")

def create_category_analysis(df):
    """Create category-level analysis"""
    if df.empty:
        return pd.DataFrame()

    category_stats = df.groupby('Category').agg({
        'TVL_USD': ['count', 'sum', 'mean', 'median'],
        'Market_Cap_USD': ['sum', 'mean'],
        'TVL_Change_1d': 'mean',
        'Price_Change_24h': 'mean'
    }).round(2)

    # Flatten column names
    category_stats.columns = [f"{col[0]}_{col[1]}" for col in category_stats.columns]
    category_stats = category_stats.reset_index()

    # Rename for clarity
    category_stats.rename(columns={
        'TVL_USD_count': 'Protocol_Count',
        'TVL_USD_sum': 'Total_TVL',
        'TVL_USD_mean': 'Avg_TVL',
        'TVL_USD_median': 'Median_TVL',
        'Market_Cap_USD_sum': 'Total_Market_Cap',
        'Market_Cap_USD_mean': 'Avg_Market_Cap',
        'TVL_Change_1d_mean': 'Avg_TVL_Change_1d',
        'Price_Change_24h_mean': 'Avg_Price_Change_24h'
    }, inplace=True)

    category_stats = category_stats.sort_values('Total_TVL', ascending=False)
    print(f"📊 Category analysis: {len(category_stats)} categories")
    return category_stats

def create_financial_rankings(df):
    """Create various financial rankings"""
    if df.empty:
        return {}

    rankings = {}

    # Top protocols by revenue
    if 'Revenue_24h' in df.columns:
        rankings['top_revenue'] = df[df['Revenue_24h'] > 0].nlargest(20, 'Revenue_24h')[
            ['Protocol', 'Category', 'Revenue_24h', 'Revenue_7d', 'Revenue_30d']
        ].copy()
    else:
        rankings['top_revenue'] = pd.DataFrame()

    # Top protocols by fees
    if 'Fees_24h' in df.columns:
        rankings['top_fees'] = df[df['Fees_24h'] > 0].nlargest(20, 'Fees_24h')[
            ['Protocol', 'Category', 'Fees_24h', 'Fees_7d', 'Fees_30d']
        ].copy()
    else:
        rankings['top_fees'] = pd.DataFrame()

    # Best P/F ratios (lowest = best value)
    if 'PF_Ratio' in df.columns:
        rankings['best_pf_ratios'] = df[
            (df['PF_Ratio'].notna()) & (df['PF_Ratio'] > 0) & (df['PF_Ratio'] < 100)
        ].nsmallest(15, 'PF_Ratio')[
            ['Protocol', 'Category', 'Market_Cap_USD', 'Fees_24h', 'PF_Ratio']
        ].copy()
    else:
        rankings['best_pf_ratios'] = pd.DataFrame()

    # Best P/R ratios (lowest = best value)
    if 'PR_Ratio' in df.columns:
        rankings['best_pr_ratios'] = df[
            (df['PR_Ratio'].notna()) & (df['PR_Ratio'] > 0) & (df['PR_Ratio'] < 100)
        ].nsmallest(15, 'PR_Ratio')[
            ['Protocol', 'Category', 'Market_Cap_USD', 'Revenue_24h', 'PR_Ratio']
        ].copy()
    else:
        rankings['best_pr_ratios'] = pd.DataFrame()

    print(f"📊 Created {len(rankings)} financial ranking tables")
    return rankings

def create_token_distribution_summary(df):
    """Create token distribution analysis summary"""
    if df.empty:
        return df, pd.DataFrame(), pd.DataFrame()

    # Summary statistics
    summary_stats = df.agg({
        'total_accounts_analyzed': 'mean',
        'top_1_holder_share': ['mean', 'median', 'std'],
        'top_5_holders_share': ['mean', 'median', 'std'],
        'top_10_holders_share': ['mean', 'median', 'std'],
        'gini_coefficient': ['mean', 'median', 'std']
    }).round(3)

    # Create distribution categories
    df = df.copy()
    df['concentration_level'] = pd.cut(
        df['top_10_holders_share'],
        bins=[0, 25, 50, 75, 100],
        labels=['Low (0-25%)', 'Medium (25-50%)', 'High (50-75%)', 'Very High (75-100%)']
    )

    concentration_dist = df['concentration_level'].value_counts().reset_index()
    concentration_dist.rename(columns={'index': 'Concentration_Level', 'concentration_level': 'Token_Count'}, inplace=True)

    print(f"📊 Token distribution summary: {len(df)} tokens analyzed")
    return df, summary_stats, concentration_dist

# Execute analysis
category_analysis = create_category_analysis(df_tab1)
financial_rankings = create_financial_rankings(df_tab2)
df_tab3_enhanced, distribution_summary, concentration_dist = create_token_distribution_summary(df_tab3)

print("\n💾 Saving Streamlit-Ready Datasets...")

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Save datasets for Streamlit
datasets = {
    'tab1_overview': df_tab1,
    'tab2_financial': df_tab2,
    'tab3_distribution': df_tab3_enhanced,
    'category_analysis': category_analysis,
    'financial_rankings': financial_rankings,
    'distribution_summary': distribution_summary,
    'concentration_distribution': concentration_dist,
    'full_dataset': df_full
}

for name, dataset in datasets.items():
    if isinstance(dataset, pd.DataFrame) and not dataset.empty:
        filepath = os.path.join(streamlit_data_dir, f'{name}_{timestamp}.joblib')
        joblib.dump(dataset, filepath, compress='zlib')
        print(f"💾 Saved {name}: {filepath} (Shape: {dataset.shape})")
    elif isinstance(dataset, dict) and dataset:
        filepath = os.path.join(streamlit_data_dir, f'{name}_{timestamp}.joblib')
        joblib.dump(dataset, filepath, compress='zlib')
        print(f"💾 Saved {name}: {filepath} (Dict with {len(dataset)} items)")

print("\n🎯 Creating Summary Statistics for Streamlit...")

# Create summary statistics
summary_stats = {}

if not df_tab1.empty:
    summary_stats['overview'] = {
        'total_protocols': len(df_tab1),
        'total_tvl': df_tab1['TVL_USD'].sum(),
        'avg_tvl': df_tab1['TVL_USD'].mean(),
        'total_market_cap': df_tab1['Market_Cap_USD'].sum(),
        'protocols_with_tokens': len(df_tab1[df_tab1['Market_Cap_USD'] > 0]),
        'top_category': df_tab1.groupby('Category')['TVL_USD'].sum().idxmax() if not df_tab1.empty else 'N/A',
        'most_valuable_protocol': df_tab1.loc[df_tab1['TVL_USD'].idxmax(), 'Protocol'] if not df_tab1.empty else 'N/A',
        'timestamp': datetime.now()
    }

if not df_tab2.empty:
    summary_stats['financial'] = {
        'protocols_with_revenue': len(df_tab2[df_tab2['Revenue_24h'] > 0]),
        'protocols_with_fees': len(df_tab2[df_tab2['Fees_24h'] > 0]),
        'total_daily_revenue': df_tab2['Revenue_24h'].sum(),
        'total_daily_fees': df_tab2['Fees_24h'].sum(),
        'avg_pf_ratio': df_tab2['PF_Ratio'].median(),
        'avg_pr_ratio': df_tab2['PR_Ratio'].median(),
        'highest_revenue_protocol': df_tab2.loc[df_tab2['Revenue_24h'].idxmax(), 'Protocol'] if df_tab2['Revenue_24h'].max() > 0 else 'N/A',
        'timestamp': datetime.now()
    }

if not df_tab3.empty:
    summary_stats['distribution'] = {
        'tokens_analyzed': len(df_tab3),
        'avg_gini_coefficient': df_tab3['gini_coefficient'].mean(),
        'avg_top_10_share': df_tab3['top_10_holders_share'].mean(),
        'most_concentrated_token': df_tab3.loc[df_tab3['top_10_holders_share'].idxmax(), 'token_name'] if not df_tab3.empty else 'N/A',
        'least_concentrated_token': df_tab3.loc[df_tab3['top_10_holders_share'].idxmin(), 'token_name'] if not df_tab3.empty else 'N/A',
        'timestamp': datetime.now()
    }

# Save summary statistics
summary_filepath = os.path.join(streamlit_data_dir, f'summary_stats_{timestamp}.joblib')
joblib.dump(summary_stats, summary_filepath, compress='zlib')
print(f"💾 Saved summary statistics: {summary_filepath}")

print("\n📊 Data Processing Complete!")
print("=" * 60)
print("Ready for Streamlit Application:")
print(f"  📁 Data directory: {streamlit_data_dir}")
print(f"  📊 Tab 1 (Overview): {df_tab1.shape if not df_tab1.empty else 'No data'}")
print(f"  💰 Tab 2 (Financial): {df_tab2.shape if not df_tab2.empty else 'No data'}")
print(f"  🎯 Tab 3 (Distribution): {df_tab3.shape if not df_tab3.empty else 'No data'}")

if summary_stats:
    print("\n📈 Key Metrics Summary:")
    if 'overview' in summary_stats:
        print(f"  • Total Protocols: {summary_stats['overview']['total_protocols']}")
        print(f"  • Total TVL: {format_currency(summary_stats['overview']['total_tvl'])}")
        print(f"  • Total Market Cap: {format_currency(summary_stats['overview']['total_market_cap'])}")

    if 'financial' in summary_stats:
        print(f"  • Daily Revenue: {format_currency(summary_stats['financial']['total_daily_revenue'])}")
        print(f"  • Daily Fees: {format_currency(summary_stats['financial']['total_daily_fees'])}")

    if 'distribution' in summary_stats:
        print(f"  • Tokens Analyzed: {summary_stats['distribution']['tokens_analyzed']}")
        print(f"  • Avg Gini Coefficient: {summary_stats['distribution']['avg_gini_coefficient']:.3f}")

print("\n🚀 Data is ready for Streamlit dashboard implementation!")

# Create a metadata file for easy data loading in Streamlit
metadata = {
    'last_updated': datetime.now(),
    'data_files': {
        'tab1_overview': f'tab1_overview_{timestamp}.joblib',
        'tab2_financial': f'tab2_financial_{timestamp}.joblib',
        'tab3_distribution': f'tab3_distribution_{timestamp}.joblib',
        'category_analysis': f'category_analysis_{timestamp}.joblib',
        'financial_rankings': f'financial_rankings_{timestamp}.joblib',
        'summary_stats': f'summary_stats_{timestamp}.joblib'
    },
    'record_counts': {
        'protocols_overview': len(df_tab1),
        'protocols_financial': len(df_tab2),
        'tokens_distribution': len(df_tab3)
    }
}

metadata_filepath = os.path.join(streamlit_data_dir, 'latest_data_metadata.joblib')
joblib.dump(metadata, metadata_filepath)
print(f"💾 Saved metadata: {metadata_filepath}")

print("\n" + "=" * 60)


📊 Solana DeFi Tracker - Analysis for Streamlit
Processing data for Streamlit application
Analysis timestamp: 2025-09-01 17:54:00

🔍 Loading Collected Data...
✅ Loaded TVL data: solana_defi_tvl_20250901_163640.joblib
✅ Loaded TVL data (Shape: (250, 14))
Columns: ['name', 'slug', 'tvl', 'chains', 'category', 'change_1h', 'change_1d', 'change_7d', 'mcap', 'symbol', 'url', 'description', 'gecko_id', 'timestamp']
✅ Loaded Revenue data: solana_revenue_20250901_164725.joblib
✅ Loaded Revenue data (Shape: (117, 7))
Columns: ['protocol', 'total_24h', 'total_7d', 'total_30d', 'total_all_time', 'chain', 'timestamp']
✅ Loaded Fees data: solana_fees_20250901_164810.joblib
✅ Converted Fees data to DataFrame (Shape: (117, 7))
Columns: ['protocol', 'total_24h', 'total_7d', 'total_30d', 'total_all_time', 'chain', 'timestamp']
✅ Loaded CoinGecko data: solana_coingecko_enhanced_20250901_165439.joblib
ℹ️ CoinGecko data is a dict (keeping as-is for specialized handling)
✅ Converted CoinGecko dict to DataFr